In [4]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Set the path to your dataset
train_data_dir = r'C:\Users\Aman Yadav\Downloads\organic_recycable\DATASET\DATASET\TRAIN'
test_data_dir = r'C:\Users\Aman Yadav\Downloads\organic_recycable\DATASET\DATASET\TEST'

# Image parameters
img_width, img_height = 150, 150
input_shape = (img_width, img_height, 3)
batch_size = 32

# Data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Rescale the validation and test sets
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load and augment the training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',  # 'categorical' for more than two classes
    shuffle=True
)


# Load the test data
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))  # Two output neurons for two classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

# Save the model
model.save('waste_classification_model_recyclable_organic.h5')


Found 22564 images belonging to 2 classes.
Found 2513 images belonging to 2 classes.
Epoch 1/10
705/705 [==============================] - 534s 754ms/step - loss: 0.4280 - accuracy: 0.8108
Epoch 2/10
705/705 [==============================] - 502s 712ms/step - loss: 0.3663 - accuracy: 0.8439
Epoch 3/10
705/705 [==============================] - 507s 718ms/step - loss: 0.3493 - accuracy: 0.8533
Epoch 4/10
705/705 [==============================] - 478s 678ms/step - loss: 0.3291 - accuracy: 0.8633
Epoch 5/10
705/705 [==============================] - 469s 665ms/step - loss: 0.3121 - accuracy: 0.8728
Epoch 6/10
705/705 [==============================] - 470s 666ms/step - loss: 0.2997 - accuracy: 0.8768
Epoch 7/10
705/705 [==============================] - 467s 662ms/step - loss: 0.2875 - accuracy: 0.8823
Epoch 8/10
705/705 [==============================] - 460s 652ms/step - loss: 0.2733 - accuracy: 0.8899
Epoch 9/10
705/705 [==============================] - 455s 644ms/step - loss: 0.261

C:\Users\Aman Yadav\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('waste_classification_model_recyclable_organic.h5')

# Function to preprocess an image for prediction
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Rescale pixel values to [0, 1]
    return img_array

# Function to predict waste type
def predict_waste_type(image_path):
    img_array = preprocess_image(image_path)
    prediction = model.predict(img_array)
    waste_type = "Organic" if prediction[0][0] > 0.5 else "Recyclable"
    confidence = prediction[0][0] if waste_type == "Organic" else 1 - prediction[0][0]
    return waste_type, confidence

# Example usage:
image_path = r'C:\Users\Aman Yadav\Downloads\organic_recycable\DATASET\DATASET\TEST\R\R_11088.jpg'
waste_type, confidence = predict_waste_type(image_path)
print(f'Predicted Waste Type: {waste_type} with confidence: {confidence:.2%}')


1/1 [==============================] - 0s 260ms/step
Predicted Waste Type: Recyclable with confidence: 98.24%
